#Итоговый проект

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from os import listdir
import codecs
import pickle                                                                   
from bs4 import BeautifulSoup as BS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from boilerpipe.extract import Extractor
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import GradientBoostingClassifier
from gensim.models import Word2Vec


from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
import re
%matplotlib inline

D:\Anaconda3\lib\site-packages\jpype\_core.py:209: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [66]:
submit = True

In [3]:
def save_obj(obj, name ):                                                       
    with open(name + '.pkl', 'wb') as f:                                        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            
                                                                                
def load_obj(name ):                                                            
    with open(name + '.pkl', 'rb') as f:                                        
        return pickle.load(f)       

In [4]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return ' '.join(res)

In [5]:
name = 'NumWordsRulesExtractor'
#name = 'ArticleExtractor'
name = 'train_data'+name
#train_data = load_obj(name)#Загружаем готовый распарсенный текст уже

In [37]:
#это парсинг теста и доп фич (meta)
if submit:
    test_data= pd.read_csv('test_groups.csv',dtype=np.int16)

    parser_type = 'NumWordsRulesExtractor'
    def save_obj(obj, name ):                                                       
        with open(name + '.pkl', 'wb') as f:                                        
            pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            

    def load_obj(name ):                                                            
        with open(name + '.pkl', 'rb') as f:                                        
            return pickle.load(f)                                                   
        
    description = {}                                                                      
    keywords= {}                                                                      
    p = {} 
    h1 = {} 
    h2 = {} 
    h3 = {} 
    h4 = {} 
    h5 = {} 
    h6 = {} 
    h1_text = {}
    h2_text = {}
    h3_text = {}
    h4_text = {}
    h5_text = {}
    h6_text = {}
    test_data['description'] = ''
    test_data['keywords'] = ''

    path = 'content/'                                                               
    texts = {}                                                                      
    print('Start: ' + parser_type)                                                  
    test_data = pd.read_csv('test_groups.csv',dtype=np.int16)                      
    #for filename in ['6770.dat']:                                                  
    for filename in tqdm(listdir(path)):                                            
        doc_id = int(filename.strip('.dat'))                                        
        if doc_id not in test_data.doc_id.values:                                  
            continue                                                                
        try:                                                                        
            with codecs.open(path + filename, 'r', encoding='utf-8', errors='ignore') as f:
                url = f.readline().strip()                                          
                html = f.read()                                                     
                bs = BS(html, 'html.parser')#ищем след страницу                         
                meta = bs.find("meta", {"name":"description"})
                if meta:
                    description[doc_id] = meta.get('content','')
                meta = bs.find("meta", {"name":"keywords"})
                if meta:
                    keywords[doc_id] = meta.get('content','')
                    
                p[doc_id] = 0 
                h1[doc_id] = 0 
                h2[doc_id] = 0 
                h3[doc_id] = 0 
                h4[doc_id] = 0 
                h5[doc_id] = 0 
                h6[doc_id] = 0 

                p[doc_id] += len(bs.find_all('p'))
                h1[doc_id] += len(bs.find_all('h1'))
                h2[doc_id] += len(bs.find_all('h2'))
                h3[doc_id] += len(bs.find_all('h3'))
                h4[doc_id] += len(bs.find_all('h4'))
                h5[doc_id] += len(bs.find_all('h5'))
                h6[doc_id] += len(bs.find_all('h6'))
                
                h1_all = bs.find_all("h1")
                h2_all = bs.find_all("h2")
                h3_all = bs.find_all("h3")
                h4_all = bs.find_all("h4")
                h5_all = bs.find_all("h5")
                h6_all = bs.find_all("h6")
            
                str = ''
                for h in h1_all:
                    s=h.text.replace('\n'," ")                                                    
                    s=s.replace('\t'," ")                                                    
                    s=s.replace('\r'," ")          
                    str+=' '+s
                h1_text[doc_id] = str
        
                str = ''
                for h in h2_all:
                    s=h.text.replace('\n'," ")                                                    
                    s=s.replace('\t'," ")                                                    
                    s=s.replace('\r'," ")          
                    str+=' '+s        
                h2_text[doc_id] = str
        
                str = ''
                for h in h3_all:
                    s=h.text.replace('\n'," ")                                                    
                    s=s.replace('\t'," ")                                                    
                    s=s.replace('\r'," ")          
                    str+=' '+s
                h3_text[doc_id] = str
        
                str = ''
                for h in h4_all:
                    s=h.text.replace('\n'," ")                                                    
                    s=s.replace('\t'," ")                                                    
                    s=s.replace('\r'," ")          
                    str+=' '+s 
                h4_text[doc_id] = str
        
                str = ''
                for h in h5_all:
                    s=h.text.replace('\n'," ")                                                    
                    s=s.replace('\t'," ")                                                    
                    s=s.replace('\r'," ")          
                    str+=' '+s
                h5_text[doc_id] = str
        
                str = ''
                for h in h6_all:
                    s=h.text.replace('\n'," ")                                                    
                    s=s.replace('\t'," ")                                                    
                    s=s.replace('\r'," ")          
                    str+=' '+s
                h6_text[doc_id] = str
                
                
                extractor = Extractor(extractor=parser_type, html=html)             
                s = extractor.getText()                                             
                s=s.replace('\n'," ")                                               
                s=s.replace('\t'," ")                                               
                s=s.replace('\r'," ")                                               
                texts[doc_id] = s                                                   
        except UnicodeDecodeError: 
            print('error {}'.format(filename))                                      
            with codecs.open(path + filename, 'r', 'utf-8') as f:                   
                url = f.readline().strip()                                          
                html = f.read()                                                     
            bs = BS(html, 'html.parser')#ищем след страницу                         
            for s in bs.select('script'):                                           
                s.extract()                                                         
            s = bs.get_text()                                                       
            s=s.replace('\n',"")                                                    
            s=s.replace('\t',"")                                                    
            s=s.replace('\r',"")                                                    
            texts[doc_id] = s                                                       
             
    test_data['p'] = test_data.apply(lambda row: p.get(row.doc_id,0), axis = 1)  
    test_data['h1'] = test_data.apply(lambda row: h1.get(row.doc_id,0), axis = 1)  
    test_data['h2'] = test_data.apply(lambda row: h2.get(row.doc_id,0), axis = 1)  
    test_data['h3'] = test_data.apply(lambda row: h3.get(row.doc_id,0), axis = 1)  
    test_data['h4'] = test_data.apply(lambda row: h4.get(row.doc_id,0), axis = 1)  
    test_data['h5'] = test_data.apply(lambda row: h5.get(row.doc_id,0), axis = 1)  
    test_data['h6'] = test_data.apply(lambda row: h6.get(row.doc_id,0), axis = 1)  
    test_data['text'] = test_data.apply(lambda row: texts[row.doc_id], axis = 1)  
    test_data['description'] = test_data.apply(lambda row: description.get(row.doc_id,''), axis = 1)  
    test_data['keywords'] = test_data.apply(lambda row: keywords.get(row.doc_id,''), axis = 1)  
    test_data['h1_text'] = test_data.apply(lambda row: h1_text.get(row.doc_id,''), axis = 1)  
    test_data['h2_text'] = test_data.apply(lambda row: h2_text.get(row.doc_id,''), axis = 1)  
    test_data['h3_text'] = test_data.apply(lambda row: h3_text.get(row.doc_id,''), axis = 1)  
    test_data['h4_text'] = test_data.apply(lambda row: h4_text.get(row.doc_id,''), axis = 1)  
    test_data['h5_text'] = test_data.apply(lambda row: h5_text.get(row.doc_id,''), axis = 1)  
    test_data['h6_text'] = test_data.apply(lambda row: h6_text.get(row.doc_id,''), axis = 1) 
else:
#     path = 'content/'                                                               
#     description = {}                                                                      
#     keywords= {}                                                                      
#     train_data['description'] = ''
#     train_data['keywords'] = ''
#     #for filename in ['1.dat']:                                                  
#     for filename in tqdm(listdir(path)):                                            
#         doc_id = int(filename.strip('.dat'))                                        
#         if doc_id not in train_data.doc_id.values:                                  
#             continue                                                                
#         with codecs.open(path + filename, 'r', encoding='utf-8', errors='ignore') as f:
#             url = f.readline().strip()                                          
#     #        urls[doc_id] = url                                                       
#             html = f.read()                                                     
#             bs = BS(html, 'html.parser')#ищем след страницу                         
#             meta = bs.find("meta", {"name":"description"})
#             if meta:
#                 description[doc_id] = lemmatize(meta.get('content',''))
#             meta = bs.find("meta", {"name":"keywords"})
#             if meta:
#                 keywords[doc_id] = lemmatize(meta.get('content',''))
#     train_data['description'] = train_data.apply(lambda row: description.get(row.doc_id,''), axis = 1)  
#     train_data['keywords'] = train_data.apply(lambda row: keywords.get(row.doc_id,''), axis = 1)  
    path = 'content/'                                                               
    p = {} 
    h1 = {} 
    h2 = {} 
    h3 = {} 
    h4 = {} 
    h5 = {} 
    h6 = {} 
    h1_text = {}
    h2_text = {}
    h3_text = {}
    h4_text = {}
    h5_text = {}
    h6_text = {}
    #urls = {}                                                                      
#     hrefs_n = {}                                                                      
#     hrefs = {}                                                                      
    description = {}                                                                      
    keywords= {}                                                                      
    train_data['description'] = ''
    train_data['keywords'] = ''
#     train_data['url'] = ''
#     train_data['hrefs'] = ''
#     train_data['hrefs_n'] = 0
#    for filename in ['15731.dat']:                                                  
    for filename in tqdm(listdir(path)):                                            
        doc_id = int(filename.strip('.dat'))                                        
        if doc_id not in train_data.doc_id.values:                                  
            continue                                                                
        with codecs.open(path + filename, 'r', encoding='utf-8', errors='ignore') as f:
            url = f.readline().strip()                                          
            #urls[doc_id] = re.split(';|,|\.|\_|\-|\*|\/|\&|\?|\=',url)
            html = f.read()                                                     
            bs = BS(html, 'html.parser')#ищем след страницу                         
            meta = bs.find("meta", {"name":"description"})
            if meta:
                description[doc_id] = lemmatize(meta.get('content',''))
            meta = bs.find("meta", {"name":"keywords"})
            if meta:
                 keywords[doc_id] = lemmatize(meta.get('content',''))
#             hrefs_n[doc_id] = 0 
#             hrefs[doc_id] = []                                                       
           
#             hrefs_bs = bs.find_all(href=True)
#             for href in hrefs_bs:
#                 if href:
#                     if href['href'][0:4] == 'http':
#                         hrefs[doc_id].append(re.split(';|,|\.|\_|\-|\*|\/|\&|\?|\=',href['href']))
#                         hrefs_n[doc_id] +=1
            p[doc_id] = 0 
            h1[doc_id] = 0 
            h2[doc_id] = 0 
            h3[doc_id] = 0 
            h4[doc_id] = 0 
            h5[doc_id] = 0 
            h6[doc_id] = 0 
            
            p[doc_id] += len(bs.find_all('p'))
            h1[doc_id] += len(bs.find_all('h1'))
            h2[doc_id] += len(bs.find_all('h2'))
            h3[doc_id] += len(bs.find_all('h3'))
            h4[doc_id] += len(bs.find_all('h4'))
            h5[doc_id] += len(bs.find_all('h5'))
            h6[doc_id] += len(bs.find_all('h6'))
            h1_all = bs.find_all("h1")
            h2_all = bs.find_all("h2")
            h3_all = bs.find_all("h3")
            h4_all = bs.find_all("h4")
            h5_all = bs.find_all("h5")
            h6_all = bs.find_all("h6")
    
            str = ''
            for h in h1_all:
                s=h.text.replace('\n'," ")                                                    
                s=s.replace('\t'," ")                                                    
                s=s.replace('\r'," ")          
                str+=' '+s
            h1_text[doc_id] = str
        
            str = ''
            for h in h2_all:
                s=h.text.replace('\n'," ")                                                    
                s=s.replace('\t'," ")                                                    
                s=s.replace('\r'," ")          
                str+=' '+s        
            h2_text[doc_id] = str
        
            str = ''
            for h in h3_all:
                s=h.text.replace('\n'," ")                                                    
                s=s.replace('\t'," ")                                                    
                s=s.replace('\r'," ")          
                str+=' '+s
            h3_text[doc_id] = str
        
            str = ''
            for h in h4_all:
                s=h.text.replace('\n'," ")                                                    
                s=s.replace('\t'," ")                                                    
                s=s.replace('\r'," ")          
                str+=' '+s 
            h4_text[doc_id] = str
        
            str = ''
            for h in h5_all:
                s=h.text.replace('\n'," ")                                                    
                s=s.replace('\t'," ")                                                    
                s=s.replace('\r'," ")          
                str+=' '+s
            h5_text[doc_id] = str
        
            str = ''
            for h in h6_all:
                s=h.text.replace('\n'," ")                                                    
                s=s.replace('\t'," ")                                                    
                s=s.replace('\r'," ")          
                str+=' '+s
            h6_text[doc_id] = str
             
    train_data['p'] = train_data.apply(lambda row: p.get(row.doc_id,0), axis = 1)  
    train_data['h1'] = train_data.apply(lambda row: h1.get(row.doc_id,0), axis = 1)  
    train_data['h2'] = train_data.apply(lambda row: h2.get(row.doc_id,0), axis = 1)  
    train_data['h3'] = train_data.apply(lambda row: h3.get(row.doc_id,0), axis = 1)  
    train_data['h4'] = train_data.apply(lambda row: h4.get(row.doc_id,0), axis = 1)  
    train_data['h5'] = train_data.apply(lambda row: h5.get(row.doc_id,0), axis = 1)  
    train_data['h6'] = train_data.apply(lambda row: h6.get(row.doc_id,0), axis = 1)
    train_data['description'] = train_data.apply(lambda row: description.get(row.doc_id,''), axis = 1)  
    train_data['h1_text'] = train_data.apply(lambda row: h1_text.get(row.doc_id,''), axis = 1)  
    train_data['h2_text'] = train_data.apply(lambda row: h2_text.get(row.doc_id,''), axis = 1)  
    train_data['h3_text'] = train_data.apply(lambda row: h3_text.get(row.doc_id,''), axis = 1)  
    train_data['h4_text'] = train_data.apply(lambda row: h4_text.get(row.doc_id,''), axis = 1)  
    train_data['h5_text'] = train_data.apply(lambda row: h5_text.get(row.doc_id,''), axis = 1)  
    train_data['h6_text'] = train_data.apply(lambda row: h6_text.get(row.doc_id,''), axis = 1)   
    train_data['keywords'] = train_data.apply(lambda row: keywords.get(row.doc_id,''), axis = 1)  

                
#     train_data['url'] = train_data.apply(lambda row: urls.get(row.doc_id,''), axis = 1)  
#     train_data['hrefs'] = train_data.apply(lambda row: hrefs.get(row.doc_id,''), axis = 1)  
#     train_data['hrefs_n'] = train_data.apply(lambda row: hrefs_n.get(row.doc_id,''), axis = 1)  
#     train_data['description'] = train_data.apply(lambda row: description.get(row.doc_id,''), axis = 1)  
#     train_data['keywords'] = train_data.apply(lambda row: keywords.get(row.doc_id,''), axis = 1)  


Start: NumWordsRulesExtractor


error 6770.dat



In [38]:
test_data.head()

,pair_id,group_id,doc_id,p,h1,h2,h3,h4,h5,h6,text,description,keywords,h1_text,h2_text,h3_text,h4_text,h5_text,h6_text
0,11691,130,6710,19,2,1,7,4,0,0,Нет Загрузка... В данный момент эта функция не...,Я в ВК http://vk.com/maksim.dolgan Обращаться ...,"Как прописать админку для кс 1.6, Как дать себ...",Это видео недоступно. ...,Текст видео,Очередь просмотра Очередь Хотите сохраните эт...,Игра Категория ...,,
1,11692,130,4030,16,10,1,0,0,0,0,Преимущества: + возможность восстановить забыт...,"Готовые сервера для SAMP 0.3.7, 0.3z, Всё для ...","В Сампе, Всампе, Самп, В-Сампе, В-Сампе.Ру, SA...",V-SAMPE.RU Топ пользователей сайта Реклама По...,Все для SAMP и GTA,,,,
2,11693,130,5561,13,4,12,1,0,0,0,"Не находит сервера в CS? Не редко бывает, что ...",,,Не находит сервера в CS? Как спалить читера в...,Не находит сервера в CS? Как спалить читера в...,Комментарии:,,,
3,11694,130,4055,3,1,0,0,0,0,0,как прописать простую админку в кс 1 6 7 ноя 2...,7 ноя 2013 Как прописать админку в cs 1.6 Чтоб...,"как, прописать, простую, админку, в, кс, 1, 6",как прописать простую админку в кс 1...,,,,,
4,11695,130,4247,1,0,0,0,0,0,0,Всем доброго времени суток. Желание админить и...,[Архив] Подбор админов для сервера по КОД_4 C...,"Подбор админов для сервера по КОД_4, сервер, ...",,,,,,


In [40]:
doc_to_title = {}
with open('docs_titles.tsv', encoding = 'UTF-8') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
if submit:
    test_doc_titles = []
    for id in test_data.doc_id.values:
        test_doc_titles.append(doc_to_title[id])
    test_data['title'] = test_doc_titles
    print(len(test_doc_titles))
else:
    doc_titles = []
    for id in train_data.doc_id.values:
        doc_titles.append(doc_to_title[id])
    train_data['title'] = doc_titles
    print(len(doc_titles))


16627


In [41]:
if submit:
    test_data['lem_title'] = test_data.apply(lambda row:lemmatize(row.title) , axis = 1)  
else:
    train_data['lem_title'] = train_data.apply(lambda row:lemmatize(row.title) , axis = 1)  

In [42]:
if submit:
    test_data['sum_title'] = test_data.apply(lambda row: row.lem_title +row.description + row.keywords , axis = 1)  
else:
    train_data['sum_title'] = train_data.apply(lambda row: row.lem_title +row.description + row.keywords , axis = 1)  


In [43]:
if submit:
    test_data['sum_headers'] = test_data.apply(lambda row: row.h1_text +row.h2_text + row.h3_text +row.h4_text + row.h5_text + row.h6_text, axis = 1)  
else:
    train_data['sum_headers'] = train_data.apply(lambda row: row.h1_text +row.h2_text + row.h3_text +row.h4_text + row.h5_text + row.h6_text, axis = 1)  
    

In [23]:
save_obj(train_data, 'train_data_headers_knn')

In [44]:
save_obj(test_data, 'test_data_headers_knn')

In [54]:
train_data = load_obj('train_data_headers_knn')

In [67]:
test_data = load_obj('test_data_headers_knn')

In [73]:
train_data

,pair_id,group_id,doc_id,target,text,p,h1,h2,h3,h4,...,h3_text,h4_text,h5_text,h6_text,title,lem_title,description,keywords,sum_title,sum_headers
0,1,1,15731,0,Сервисное обслуживание и эксплуатация Отсоедин...,48,1,0,1,0,...,ВАЗ 21213,,,,ВАЗ 21213 | Замена подшипников ступицы | Нива,ваза 21213 | замена подшипник ступица | нива,"замена подшипник ступицы, руководство по ремон...","замена подшипник ступицы, технический описание...",ваза 21213 | замена подшипник ступица | ниваза...,16.13 Замена подшипников ступицы ВАЗ 21213
1,2,1,14829,0,Ваз 2107 оптом в Сочи Смотрите также г. Толья...,40,1,0,1,0,...,Популярные категории,,,,"Ваз 2107 оптом в Сочи. Сравнить цены, купить п...","ваза 2107 оптом в сочи. сравнить цены, купить ...","ваза 2107 оптом. продажа, поиск, поставщик и м...","ваза 2107 оптом, россия, сочи, цены, предложен...","ваза 2107 оптом в сочи. сравнить цены, купить ...",Ваз 2107 оптом в Сочи Популярн...
2,3,1,15764,0,Все города ступица Показаны наиболее подходящи...,3,1,0,1,0,...,Запчасть,,,,Купить ступица Лада калина2. Трансмиссия - пер...,купить ступица лада калина2. трансмиссия - пер...,продажа запчасть ступица для легковой и грузов...,"цена, замена, тюнинг, купить, продать, ступица...",купить ступица лада калина2. трансмиссия - пер...,Ступица Лада калина2 ...
3,4,1,17669,0,"Классика 21010 - 21074 121,00 121,00 539,00 57...",0,1,2,0,0,...,,,,,Классика 21010 - 21074,классика 21010 - 21074,,,классика 21010 - 21074,Learn Center Классика 21010 - 21074 Прайс
4,5,1,14852,0,Содержание статьи: Ступица Нива переднего коле...,11,1,0,3,0,...,Устройство и особенности cтупицы переднего ко...,,,,Ступица Нива — замена подшипника своими руками,ступица нива — замена подшипник свой рука,передний ступица нива иметь свой особенность в...,,ступица нива — замена подшипник свой рукаперед...,Ступица Нива — как провести ремонт и замену п...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11685,11686,129,26672,0,"25.03.2014 Котейка Мурысовна, мальчик Миша 335...",0,0,0,1,0,...,❤★✿★АПРЕЛЯТА 2014 -6❤★✿,,,,❤★✿★АПРЕЛЯТА 2014 -6❤★✿,❤★✿★апрелёнок 2014 -6❤★✿,,,❤★✿★апрелёнок 2014 -6❤★✿,❤★✿★АПРЕЛЯТА 2014 -6❤★✿
11686,11687,129,25838,0,Geschrieben von Romana am 05.05.2012 Schöne Ho...,0,0,1,0,1,...,,Wir freuen uns über Ihren Kommentar,Antwort schreiben,,:: Gästebuch,:: gästebuch,,,:: gästebuch,links: Wir freuen uns über Ihren Kommentar An...
11687,11688,129,25703,0,"РЕБЯЯЯТА!! / ЗАХОДИМ И ПОМОГАЕМ!, Реферат ф фи...",398,51,51,47,0,...,Готовимся к chgk Таганрог (СКЦ “Приморский”) ...,,,,Jizolofej: Archive,jizolofej: archive,,,jizolofej: archive,Jizolofej Философия ф бэкона реферат Кaртошкa...
11688,11689,129,27885,0,1 апреля прошлого года 21-летний Сергей Семе...,0,1,36,0,0,...,,,,,Как зовут парня дианы шурыгиной | Пусть говоря...,как звать парень диана шурыгин | пусть говорят...,,,как звать парень диана шурыгин | пусть говорят...,Как зовут парня дианы шурыгиной Пусть говорят...


In [42]:
def get_splits(data, n=3, random_state=None):
    groups = train_data.group_id.unique()
    kf = KFold(n_splits=n, shuffle=True, random_state=random_state)
    for i,j in kf.split(groups):
        train, test = data.group_id.isin(groups[i].reshape(-1)), data.group_id.isin(groups[j].reshape(-1))
        yield (train[train].index.values, test[test].index.values)

In [68]:
vectorizer = CountVectorizer()#max_features = 70)
#stop_words =['без','в','во','для','до','за','И','из','к','ко','меж','на','над','о','об','обо','от','ото','по','под','при','про','с', 'а','же','и','ибо','или','как','но','раз','чем','что','не','ни','ли','ль','вот','вон','то','ну','как','же','ни','уж','ну','я', 'ты', 'он', 'мы', 'вы', 'её', 'их','кто','тот']

 
#vectorizer = TfidfVectorizer(preprocessor=lemmatize,stop_words=stop_words)#max_features = 70)
#vectorizer = CountVectorizer()#max_features = 70)
#vectorizer = TfidfVectorizer(stop_words=stop_words)#max_features = 70)
#vectorizer = TfidfVectorizer()#max_features = 70)

In [69]:
all_groups = np.array(list(range(1,310))).reshape(-1,1)
i = OneHotEncoder()
i.fit(all_groups)
if submit:
    groups_one_hot_test   = i.transform(test_data.group_id.values.reshape(-1,1))
else:
    groups_one_hot_train = i.transform(train_data.group_id.values.reshape(-1,1))
train_data.group_id.values.reshape(-1,1)

array([[  1],
       [  1],
       [  1],
       ...,
       [129],
       [129],
       [129]], dtype=int16)

In [ ]:
def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.

    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    vec = CountVectorizer(stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

    return [i[0] for i in words_freq[:n]]

In [70]:
def get_features(data):
    scaler = StandardScaler(with_mean=False)
    number = 1
    data['first_pop'] =0
    for group in tqdm(data.group_id.unique()):
        number +=1
        elements_in_same_group = data.group_id == group
        #words = data.title[elements_in_same_group] + data.text[elements_in_same_group]
        words = data.lem_title[elements_in_same_group] #data.lem_title[elements_in_same_group]
        vectorizer.fit(words)
        #chit_themes = data.new_meta[elements_in_same_group].values
        #full_string = ''
        #for i in chit_themes: 
       #     full_string = full_string+' '+i
        #most_popular_word = Counter(full_string.split()).most_common()[0:5]
        #most_popular_word = [i[0] for i in most_popular_word ]
        #features_chit_themes = np.zeros((chit_themes.shape[0],5))
        
        #for j, word in enumerate(most_popular_word):
        #    for i, val in enumerate(chit_themes):
          #      if word in val:
         #           features_chit_themes[i][j] = 1     
        #print(features_chit_themes)
        texts_vectorized_titles = vectorizer.transform(data.lem_title[elements_in_same_group])
        texts_vectorized_titles = scaler.fit_transform(texts_vectorized_titles)
        texts_vectorized_texts = vectorizer.transform(data.text[elements_in_same_group])
        texts_vectorized_texts = scaler.fit_transform(texts_vectorized_texts)
        texts_vectorized_keywords = vectorizer.transform(data.keywords[elements_in_same_group])
        texts_vectorized_keywords = scaler.fit_transform(texts_vectorized_keywords)
        texts_vectorized_description= vectorizer.transform(data.description[elements_in_same_group])
        texts_vectorized_description= scaler.fit_transform(texts_vectorized_description)
    
        texts_vectorized_sumtitles = vectorizer.transform(data.sum_title[elements_in_same_group])
        texts_vectorized_sumtitles = scaler.fit_transform(texts_vectorized_sumtitles)
        
        texts_vectorized_headers = vectorizer.transform(data.sum_headers[elements_in_same_group])
        texts_vectorized_headers = scaler.fit_transform(texts_vectorized_headers)
        
# #        vectorizer2.fit(data.new_meta[elements_in_same_group])
#        texts_vectorized_new_meta= vectorizer2.transform(data.new_meta[elements_in_same_group])
#        texts_vectorized_new_meta = csr_matrix(np.abs(texts_vectorized_new_meta.toarray()-1))
#        texts_vectorized_new_meta= scaler.fit_transform(texts_vectorized_new_meta)
#         urls = data.url[elements_in_same_group] + data.hrefs[elements_in_same_group]
#         urls = data.hrefs[elements_in_same_group]
# #        print(urls)
#         vectorizer2 = TfidfVectorizer()#max_features = 70)
#         vectorizer2.fit(urls)
#         texts_vectorized_urls = vectorizer2.transform(urls)
#         texts_vectorized_urls = scaler.fit_transform(texts_vectorized_urls)
        
        dist_matr = pairwise_distances(texts_vectorized_titles, texts_vectorized_titles, metric = 'cosine')
        dist_matr2 = pairwise_distances(texts_vectorized_texts, texts_vectorized_texts, metric = 'cosine')
        dist_matr3 = pairwise_distances(texts_vectorized_texts, texts_vectorized_titles, metric = 'cosine')
        dist_matr4 = pairwise_distances(texts_vectorized_keywords, texts_vectorized_keywords, metric = 'cosine')
        dist_matr5 = pairwise_distances(texts_vectorized_description, texts_vectorized_description, metric = 'cosine')
        dist_matr6 = pairwise_distances(texts_vectorized_sumtitles, texts_vectorized_texts, metric = 'cosine')
        dist_matr7 = pairwise_distances(texts_vectorized_keywords, texts_vectorized_titles, metric = 'cosine')
        dist_matr8 = pairwise_distances(texts_vectorized_description, texts_vectorized_titles, metric = 'cosine')
        dist_matr9 = pairwise_distances(texts_vectorized_headers, texts_vectorized_headers, metric = 'cosine')

        #clustering = DBSCAN(eps = 1.3, min_samples=5).fit(dist_matr)
        #clf = IsolationForest(random_state=0).fit_predict(dist_matr)

        dist_matr_sorted = np.sort(dist_matr,axis=1)[:,0:21]
        dist_matr_sorted2 = np.sort(dist_matr2,axis=1)[:,0:21]
        dist_matr_sorted3 = np.sort(dist_matr3,axis=1)[:,0:21]
        dist_matr_sorted4 = np.sort(dist_matr4,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted5 = np.sort(dist_matr5,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted6 = np.sort(dist_matr6,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted7 = np.sort(dist_matr7,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted8 = np.sort(dist_matr8,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted9 = np.sort(dist_matr9,axis=1)[:,:-1][:,0:21]

#         dist_matr_sorted = np.sort(dist_matr,axis=1)[:,0:min(50,dist_matr.shape[1])]
#         dist_matr_sorted2 = np.sort(dist_matr2,axis=1)[:,0:min(50,dist_matr2.shape[1])]
#         dist_matr_sorted3 = np.sort(dist_matr3,axis=1)[:,0:min(50,dist_matr3.shape[1])]

#         if dist_matr_sorted.shape[1] < 50:
#             dist_matr_sorted = np.hstack((dist_matr_sorted, np.tile(dist_matr_sorted[:,-1:], (1, 50-dist_matr_sorted.shape[1]))))
#             dist_matr_sorted2 = np.hstack((dist_matr_sorted2, np.tile(dist_matr_sorted2[:,-1:], (1, 50-dist_matr_sorted2.shape[1]))))
#             dist_matr_sorted3 = np.hstack((dist_matr_sorted3, np.tile(dist_matr_sorted3[:,-1:], (1, 50-dist_matr_sorted3.shape[1]))))


        mean_dist = dist_matr.mean(axis = 1)
        mean_dist2 = dist_matr2.mean(axis = 1)
        mean_dist3 = dist_matr3.mean(axis = 1)
        mean_dist4 = dist_matr4.mean(axis = 1)
        mean_dist5 = dist_matr5.mean(axis = 1)
        mean_dist6 = dist_matr6.mean(axis = 1)
        mean_dist7 = dist_matr7.mean(axis = 1)
        mean_dist8 = dist_matr8.mean(axis = 1)
        mean_dist9 = dist_matr9.mean(axis = 1)

        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist2.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist3.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist4.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist5.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist6.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist7.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist8.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist9.reshape(-1,1)))

#         pop_words = get_top_n_words(words,n=100)
#         for pop_word in pop_words:
#             data.loc[elements_in_same_group,'first_pop']= data[elements_in_same_group].apply(lambda row: int(pop_word in row.text) , axis = 1)  
#             pop  = csr_matrix(data[elements_in_same_group].first_pop.values)
#             print(pop_word,pop.toarray())
#             dist_matr_sorted = np.hstack((dist_matr_sorted, pop.toarray().reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted2))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted3))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted4))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted5))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted6))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted7))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted8))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted9))
        #dist_matr_sorted = np.hstack((dist_matr_sorted, features_chit_themes))

        #n =  data.hrefs_n[elements_in_same_group].values
        #dist_matr_sorted = np.hstack((dist_matr_sorted, n.reshape(-1,1)))
        n =  data.p[elements_in_same_group].values
        dist_matr_sorted = np.hstack((dist_matr_sorted, n.reshape(-1,1)))
        n =  data.h1[elements_in_same_group].values
        dist_matr_sorted = np.hstack((dist_matr_sorted, n.reshape(-1,1)))
        n =  data.h2[elements_in_same_group].values
        dist_matr_sorted = np.hstack((dist_matr_sorted, n.reshape(-1,1)))
        n =  data.h3[elements_in_same_group].values
        dist_matr_sorted = np.hstack((dist_matr_sorted, n.reshape(-1,1)))
        n =  data.h4[elements_in_same_group].values
        dist_matr_sorted = np.hstack((dist_matr_sorted, n.reshape(-1,1)))
        n =  data.h5[elements_in_same_group].values
        dist_matr_sorted = np.hstack((dist_matr_sorted, n.reshape(-1,1)))
        n =  data.h6[elements_in_same_group].values
        dist_matr_sorted = np.hstack((dist_matr_sorted, n.reshape(-1,1)))
# #        dist_matr_sorted = np.hstack((dist_matr_sorted, clustering.labels_.reshape(-1,1)))
      #  dist_matr_sorted = np.hstack((dist_matr_sorted, texts_vectorized_new_meta.toarray()))

        if number == 2 :
            features_by_dist =  dist_matr_sorted
        else:
            features_by_dist = vstack((features_by_dist, dist_matr_sorted))
    return features_by_dist.tocsr()
                   

In [71]:
if submit:
    test_groups_feat = pd.read_csv('knn_feat_5000_t.csv')
    feat = np.array(list(test_groups_feat['feat']))
    feat = feat.reshape(test_data.shape[0],1)
else:
    train_groups_feat = pd.read_csv('knn_feat_5000.csv')
    feat = np.array(list(train_groups_feat['feat']))
    feat = feat.reshape(train_data.shape[0],1)


In [72]:
if submit:
    features_by_dist = get_features(test_data)
    features_by_dist = hstack([features_by_dist, groups_one_hot_test])
    features_by_dist = features_by_dist.tocsr()
else:
    features_by_dist = get_features(train_data)
    features_by_dist = hstack([features_by_dist, groups_one_hot_train])
    features_by_dist = features_by_dist.tocsr()
    target = train_data.target.values

In [73]:
features_by_dist = hstack([features_by_dist, feat])
features_by_dist =  features_by_dist.tocsr()

In [74]:
features_by_dist.shape

(16627, 515)

In [63]:
#VALIDATION
import xgboost
from catboost import CatBoostClassifier
import lightgbm as ltb
max_score = 0
scores = []
for i in tqdm(range(10)):
    for train, test in get_splits(train_data,n=3,random_state=i):
        #model1 = xgboost.XGBClassifier(n_estimators=200, n_jobs=-1)
        #model4 = ltb.LGBMClassifier(n_estimators=1000, n_jobs=-1)
        #model2 = RandomForestClassifier(n_jobs=6,n_estimators=1000)
        #model3 = CatBoostClassifier(n_estimators=1000)
        model5 = GradientBoostingClassifier(learning_rate=0.02,n_estimators=200, max_depth=9)
        #model1.fit(features_by_dist[train], target[train])
        #model2.fit(features_by_dist[train], target[train])
        #model3.fit(features_by_dist[train], target[train])
        #model4.fit(features_by_dist[train], target[train])
        model5.fit(features_by_dist[train], target[train])
        #pred1 = (model1.predict(features_by_dist[test]))
        #pred2 = (model2.predict(features_by_dist[test]))
        #pred3 = (model3.predict(features_by_dist[test]))
        #pred4 = (model4.predict(features_by_dist[test]))
        pred = (model5.predict(features_by_dist[test]))

        #pred = (pred1 + pred2 + pred3 + pred4 +pred5) > 2 
        #pred = pred.astype(int)
        score = f1_score(pred,target[test])
        scores.append(score)
        print(score, end=' ')
    print()

print('\nAvg score:' ,sum(scores)/len(scores)) 

0.7543194268857986 

KeyboardInterrupt: 

In [65]:
#SAVE model
from sklearn.ensemble import RandomForestClassifier 
model1 = RandomForestClassifier(n_jobs=6,n_estimators=2000)
model1.fit(features_by_dist, target)
model2 = xgboost.XGBClassifier(n_estimators=200, n_jobs=-1, learning_rate=0.032)
model2.fit(features_by_dist, target)
model3 = CatBoostClassifier(n_estimators=1000)
model3.fit(features_by_dist, target)
model4 = ltb.LGBMClassifier(n_estimators=1000, n_jobs=-1, learning_rate=0.0058)
model4.fit(features_by_dist, target)
model5 = GradientBoostingClassifier(learning_rate=0.02,n_estimators=200, max_depth=9)
model5.fit(features_by_dist, target)

save_obj(model1, 'model1')
save_obj(model2, 'model2')
save_obj(model3, 'model3')
save_obj(model4, 'model4')
save_obj(model5, 'model5')

Learning rate set to 0.029437
0:	learn: 0.6689673	total: 69.4ms	remaining: 1m 9s
1:	learn: 0.6465965	total: 130ms	remaining: 1m 4s
2:	learn: 0.6256783	total: 189ms	remaining: 1m 2s
3:	learn: 0.6075258	total: 248ms	remaining: 1m 1s
4:	learn: 0.5898392	total: 310ms	remaining: 1m 1s
5:	learn: 0.5711027	total: 372ms	remaining: 1m 1s
6:	learn: 0.5551023	total: 433ms	remaining: 1m 1s
7:	learn: 0.5399689	total: 502ms	remaining: 1m 2s
8:	learn: 0.5259947	total: 561ms	remaining: 1m 1s
9:	learn: 0.5136809	total: 620ms	remaining: 1m 1s
10:	learn: 0.5010902	total: 684ms	remaining: 1m 1s
11:	learn: 0.4897476	total: 742ms	remaining: 1m 1s
12:	learn: 0.4803614	total: 799ms	remaining: 1m
13:	learn: 0.4703510	total: 859ms	remaining: 1m
14:	learn: 0.4612398	total: 917ms	remaining: 1m
15:	learn: 0.4534583	total: 976ms	remaining: 1m
16:	learn: 0.4449682	total: 1.04s	remaining: 59.9s
17:	learn: 0.4374131	total: 1.1s	remaining: 59.9s
18:	learn: 0.4300792	total: 1.16s	remaining: 59.8s
19:	learn: 0.4239694	to

161:	learn: 0.2824983	total: 9.8s	remaining: 50.7s
162:	learn: 0.2822422	total: 9.86s	remaining: 50.6s
163:	learn: 0.2820298	total: 9.94s	remaining: 50.7s
164:	learn: 0.2818121	total: 10s	remaining: 50.6s
165:	learn: 0.2815705	total: 10.1s	remaining: 50.6s
166:	learn: 0.2813615	total: 10.1s	remaining: 50.5s
167:	learn: 0.2810829	total: 10.2s	remaining: 50.5s
168:	learn: 0.2808033	total: 10.3s	remaining: 50.5s
169:	learn: 0.2805052	total: 10.3s	remaining: 50.5s
170:	learn: 0.2802603	total: 10.4s	remaining: 50.4s
171:	learn: 0.2799940	total: 10.5s	remaining: 50.4s
172:	learn: 0.2797425	total: 10.5s	remaining: 50.4s
173:	learn: 0.2794919	total: 10.6s	remaining: 50.3s
174:	learn: 0.2792995	total: 10.7s	remaining: 50.3s
175:	learn: 0.2789509	total: 10.7s	remaining: 50.2s
176:	learn: 0.2786201	total: 10.8s	remaining: 50.1s
177:	learn: 0.2784257	total: 10.8s	remaining: 50.1s
178:	learn: 0.2782390	total: 10.9s	remaining: 50s
179:	learn: 0.2780147	total: 11s	remaining: 49.9s
180:	learn: 0.27776

323:	learn: 0.2484106	total: 19.4s	remaining: 40.4s
324:	learn: 0.2482391	total: 19.4s	remaining: 40.4s
325:	learn: 0.2481007	total: 19.5s	remaining: 40.3s
326:	learn: 0.2478796	total: 19.6s	remaining: 40.2s
327:	learn: 0.2476838	total: 19.6s	remaining: 40.2s
328:	learn: 0.2474748	total: 19.7s	remaining: 40.1s
329:	learn: 0.2472903	total: 19.7s	remaining: 40s
330:	learn: 0.2470838	total: 19.8s	remaining: 40s
331:	learn: 0.2469057	total: 19.8s	remaining: 39.9s
332:	learn: 0.2466781	total: 19.9s	remaining: 39.8s
333:	learn: 0.2465274	total: 19.9s	remaining: 39.8s
334:	learn: 0.2463101	total: 20s	remaining: 39.7s
335:	learn: 0.2460970	total: 20.1s	remaining: 39.6s
336:	learn: 0.2459000	total: 20.1s	remaining: 39.6s
337:	learn: 0.2457318	total: 20.2s	remaining: 39.5s
338:	learn: 0.2455187	total: 20.2s	remaining: 39.4s
339:	learn: 0.2452917	total: 20.3s	remaining: 39.4s
340:	learn: 0.2451482	total: 20.3s	remaining: 39.3s
341:	learn: 0.2449871	total: 20.4s	remaining: 39.2s
342:	learn: 0.2448

484:	learn: 0.2184164	total: 28.6s	remaining: 30.4s
485:	learn: 0.2182776	total: 28.7s	remaining: 30.3s
486:	learn: 0.2180742	total: 28.8s	remaining: 30.3s
487:	learn: 0.2179138	total: 28.8s	remaining: 30.2s
488:	learn: 0.2177097	total: 28.9s	remaining: 30.2s
489:	learn: 0.2175499	total: 28.9s	remaining: 30.1s
490:	learn: 0.2173897	total: 29s	remaining: 30.1s
491:	learn: 0.2172171	total: 29.1s	remaining: 30s
492:	learn: 0.2170483	total: 29.1s	remaining: 29.9s
493:	learn: 0.2168546	total: 29.2s	remaining: 29.9s
494:	learn: 0.2167019	total: 29.2s	remaining: 29.8s
495:	learn: 0.2165308	total: 29.3s	remaining: 29.8s
496:	learn: 0.2163472	total: 29.4s	remaining: 29.7s
497:	learn: 0.2161883	total: 29.4s	remaining: 29.6s
498:	learn: 0.2160205	total: 29.5s	remaining: 29.6s
499:	learn: 0.2158112	total: 29.5s	remaining: 29.5s
500:	learn: 0.2156698	total: 29.6s	remaining: 29.5s
501:	learn: 0.2155147	total: 29.6s	remaining: 29.4s
502:	learn: 0.2153704	total: 29.7s	remaining: 29.4s
503:	learn: 0.21

645:	learn: 0.1936968	total: 38.2s	remaining: 20.9s
646:	learn: 0.1935637	total: 38.3s	remaining: 20.9s
647:	learn: 0.1934242	total: 38.3s	remaining: 20.8s
648:	learn: 0.1932977	total: 38.4s	remaining: 20.8s
649:	learn: 0.1931453	total: 38.5s	remaining: 20.7s
650:	learn: 0.1930444	total: 38.5s	remaining: 20.6s
651:	learn: 0.1929516	total: 38.6s	remaining: 20.6s
652:	learn: 0.1928216	total: 38.6s	remaining: 20.5s
653:	learn: 0.1926899	total: 38.7s	remaining: 20.5s
654:	learn: 0.1925693	total: 38.8s	remaining: 20.4s
655:	learn: 0.1924614	total: 38.8s	remaining: 20.4s
656:	learn: 0.1922943	total: 38.9s	remaining: 20.3s
657:	learn: 0.1921768	total: 38.9s	remaining: 20.2s
658:	learn: 0.1920478	total: 39s	remaining: 20.2s
659:	learn: 0.1919387	total: 39s	remaining: 20.1s
660:	learn: 0.1918341	total: 39.1s	remaining: 20.1s
661:	learn: 0.1916809	total: 39.2s	remaining: 20s
662:	learn: 0.1915475	total: 39.2s	remaining: 19.9s
663:	learn: 0.1914017	total: 39.3s	remaining: 19.9s
664:	learn: 0.1912

805:	learn: 0.1734826	total: 47.8s	remaining: 11.5s
806:	learn: 0.1733879	total: 47.8s	remaining: 11.4s
807:	learn: 0.1732520	total: 47.9s	remaining: 11.4s
808:	learn: 0.1731414	total: 47.9s	remaining: 11.3s
809:	learn: 0.1730279	total: 48s	remaining: 11.3s
810:	learn: 0.1729129	total: 48s	remaining: 11.2s
811:	learn: 0.1728315	total: 48.1s	remaining: 11.1s
812:	learn: 0.1726961	total: 48.2s	remaining: 11.1s
813:	learn: 0.1725728	total: 48.2s	remaining: 11s
814:	learn: 0.1724655	total: 48.3s	remaining: 11s
815:	learn: 0.1723686	total: 48.3s	remaining: 10.9s
816:	learn: 0.1722695	total: 48.4s	remaining: 10.8s
817:	learn: 0.1721678	total: 48.5s	remaining: 10.8s
818:	learn: 0.1720679	total: 48.5s	remaining: 10.7s
819:	learn: 0.1719297	total: 48.6s	remaining: 10.7s
820:	learn: 0.1718156	total: 48.6s	remaining: 10.6s
821:	learn: 0.1717223	total: 48.7s	remaining: 10.5s
822:	learn: 0.1715939	total: 48.8s	remaining: 10.5s
823:	learn: 0.1714436	total: 48.8s	remaining: 10.4s
824:	learn: 0.171310

967:	learn: 0.1562167	total: 57.4s	remaining: 1.9s
968:	learn: 0.1561185	total: 57.4s	remaining: 1.84s
969:	learn: 0.1560159	total: 57.5s	remaining: 1.78s
970:	learn: 0.1559031	total: 57.6s	remaining: 1.72s
971:	learn: 0.1558179	total: 57.6s	remaining: 1.66s
972:	learn: 0.1557362	total: 57.7s	remaining: 1.6s
973:	learn: 0.1556250	total: 57.7s	remaining: 1.54s
974:	learn: 0.1555406	total: 57.8s	remaining: 1.48s
975:	learn: 0.1554003	total: 57.9s	remaining: 1.42s
976:	learn: 0.1553023	total: 57.9s	remaining: 1.36s
977:	learn: 0.1552058	total: 58s	remaining: 1.3s
978:	learn: 0.1551028	total: 58.1s	remaining: 1.25s
979:	learn: 0.1549821	total: 58.1s	remaining: 1.19s
980:	learn: 0.1548526	total: 58.2s	remaining: 1.13s
981:	learn: 0.1547452	total: 58.2s	remaining: 1.07s
982:	learn: 0.1546336	total: 58.3s	remaining: 1.01s
983:	learn: 0.1545432	total: 58.4s	remaining: 949ms
984:	learn: 0.1544665	total: 58.4s	remaining: 890ms
985:	learn: 0.1543691	total: 58.5s	remaining: 830ms
986:	learn: 0.154

In [75]:
#SUBMIT
from sklearn.ensemble import RandomForestClassifier 
print(type(features_by_dist))
model1 = load_obj('model1') 
model2 = load_obj('model2')
model3 = load_obj('model3')
model4 = load_obj('model4')
model5 = load_obj('model5')
pred1 = model1.predict(features_by_dist)
pred2 = model2.predict(features_by_dist)
pred3 = model3.predict(features_by_dist)
pred4 = model4.predict(features_by_dist)
pred5 = model5.predict(features_by_dist)
pred = (pred1 + pred2 + pred3 + pred4 + pred5) > 2 
pred = pred.astype(int)
pred.shape

<class 'scipy.sparse.csr.csr_matrix'>


(16627,)

In [76]:
test_data['target'] = pred
test_data.to_csv('sub_5models.csv',index=False,columns=['pair_id','target'])

In [ ]:
save_obj(test_data,'test')